# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/czh/spark-2.4.6-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Project').getOrCreate()

In [3]:
# Read in teh data
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
# Select useful features
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 
                                       'Years', 'Num_Sites'], outputCol = 'features')

output = assembler.transform(df)
data = output.select('Churn', 'features')
data.head()

Row(Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]))

In [5]:
# Split the data
train, test = data.randomSplit([0.7, 0.3], seed=13)

In [6]:
train.head(10)

[Row(Churn=0, features=DenseVector([22.0, 11254.38, 1.0, 4.96, 8.0])),
 Row(Churn=0, features=DenseVector([26.0, 8939.61, 0.0, 4.54, 7.0])),
 Row(Churn=0, features=DenseVector([27.0, 8628.8, 1.0, 5.3, 7.0])),
 Row(Churn=0, features=DenseVector([28.0, 9090.43, 1.0, 5.74, 10.0])),
 Row(Churn=0, features=DenseVector([28.0, 11128.95, 1.0, 5.12, 8.0])),
 Row(Churn=0, features=DenseVector([28.0, 11204.23, 0.0, 3.67, 11.0])),
 Row(Churn=0, features=DenseVector([28.0, 11245.38, 0.0, 6.72, 8.0])),
 Row(Churn=0, features=DenseVector([29.0, 9378.24, 0.0, 4.93, 8.0])),
 Row(Churn=0, features=DenseVector([29.0, 9617.59, 0.0, 5.49, 8.0])),
 Row(Churn=0, features=DenseVector([29.0, 10203.18, 1.0, 5.82, 8.0]))]

In [7]:
# Apply the logistic regression
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(labelCol='Churn')

model = log_reg.fit(train)

In [8]:
results_train = model.summary

In [9]:
results_train.areaUnderROC

0.8884942084942075

In [10]:
results_train.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                618|                618|
|   mean|0.16181229773462782|0.10841423948220065|
| stddev|0.36857685529640893|0.31115471133122574|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [11]:
# Evaluate on the test set
results_test = model.evaluate(test)
results_test.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[25.0,9672.03,0.0...|[4.33507425148796...|[0.98706851286089...|       0.0|
|    0|[28.0,8670.98,0.0...|[7.35934154989609...|[0.99936378752157...|       0.0|
|    0|[29.0,5900.78,1.0...|[3.52518474072085...|[0.97139591933134...|       0.0|
|    0|[30.0,6744.87,0.0...|[3.27145191633357...|[0.96343635244421...|       0.0|
|    0|[30.0,8677.28,1.0...|[3.38230199940307...|[0.96714682727525...|       0.0|
|    0|[30.0,13473.35,0....|[2.90058155050102...|[0.94787517755219...|       0.0|
|    0|[31.0,5304.6,0.0,...|[3.14996771133266...|[0.95890744951726...|       0.0|
|    0|[31.0,9574.89,0.0...|[2.83583423690373...|[0.94458180035073...|       0.0|
|    0|[31.0,10182.6,1.0...|[4.50790803830198...|[0.98909865646099...|       0.0|
|    0|[32.0,121

In [12]:
results_test.roc.show()

+--------------------+----+
|                 FPR| TPR|
+--------------------+----+
|                 0.0| 0.0|
|                 0.0|0.04|
|                 0.0|0.08|
|                 0.0|0.12|
|                 0.0|0.16|
|0.004310344827586207|0.18|
|0.004310344827586207|0.22|
|0.008620689655172414|0.24|
|0.008620689655172414|0.28|
|0.008620689655172414|0.32|
|0.017241379310344827|0.32|
| 0.02586206896551724|0.32|
| 0.02586206896551724|0.36|
| 0.03017241379310345|0.38|
|0.034482758620689655| 0.4|
|0.034482758620689655|0.44|
|0.034482758620689655|0.48|
|0.034482758620689655|0.52|
|0.034482758620689655|0.56|
| 0.04310344827586207|0.56|
+--------------------+----+
only showing top 20 rows



In [13]:
results_test.areaUnderROC

0.9387931034482767

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='Churn')

evaluator.evaluate(results_test.predictions)

0.9389655172413816

# Predict on new data

In [15]:
# Load the unlabeled data
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [16]:
predict_df = assembler.transform(new_customers)

In [17]:
# Refit the with the whole data set
finalModel = log_reg.fit(data)

In [18]:
# Predict on the new data set
predictions = finalModel.transform(predict_df)
predictions.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

